# Задание по курсу «Дискретная оптимизация», МФТИ, весна 2017

## Задача 2-2. Эвристика Кернигана—Лина

В этой задаче Вам предлагается добавить к локальному поиска в задаче о сбалансированном разбиении графа эвристику Кернигана—Лина, когда мы, «застряв» в локальном минимуме, тем не менее пытаемся сделать несколько шагов из него, даже если они приводят к временному ухудшению. Надежда здесь на то, что после ухудшения может наступить заметное улучшение результата: нам удастся выпрыгнуть из локального оптимума. Мы рассматриваем безвесовый вариант задачи о разбиении с параметром балансировки $\alpha=\frac{1}{2}$:

**Даны:**
* $G=(V,E)$ — граф без весов на рёбрах

**Найти:**
* Разбиение $V=V'\sqcup V''$, такое, что $V'=\lfloor |V|/2 \rfloor$ и число рёбер между $V'$ и $V''$ минимально возможное.

Сделайте следующее:
* Скачайте файл [`partition-instances.zip`](https://github.com/dainiak/discrete-optimization-course/raw/master/partition-instances.zip) и разархивируйте из него файлы со входами задачи.
* Для каждого из графов найдите локальным поиском с эвристикой Кернигана—Лина локально минимальное (по количеству рёбер между частями) разбиение вершин графа на две части, мощности которых отличаются не более чем на единицу. 
* Реализуйте функцию `variable_depth_local_search`; она должна принимать на вход граф в формате, предоставляемом функцией `read_instance`, и возвращать найденное разбиение как множество вершин, лежащих в одной любой из двух компонент разбиения. Ваш локальный поиск должен начинаться с того разбиение, которое уже находится в переменной `starting_point`.
* Подберите для каждого из четырёх входных графов глубину поиска так, чтобы он работал не более 60 секунд на Вашем компьютере, и сохраните информацию о подобранных параметрах и любые свои интересные наблюдения в отдельную ячейку настоящего ipynb-файла.

In [1]:
def read_instance(filename):
    with open(filename, 'r') as file:
        n_vertices = int(file.readline().strip().split()[0])
        vertices, edges = set(range(1, n_vertices + 1)), set()
        for u in range(1, n_vertices + 1):
            for v in map(int, file.readline().strip().split()):
                edges.add((u,v))
        return (vertices, edges)

In [87]:
import random
magic = 1000
magic1 = 50

def variable_depth_local_search(graph, it):
    first = list(range(1, len(graph[0]) // 2 + 1))
    second = list(set(graph[0]) - set(first))
    start = 0
    v = []
    n = len(graph[0])
    for i in range(n + 1):
        v.append([])
    edges = list(graph[1])
    for i in range(len(edges)):
        v[edges[i][0]].append(edges[i][1])
    num = [0] * (n + 1)
    for i in range(len(first)):
        num[first[i]] = 1
    for i in range(len(second)):
        num[second[i]] = 2
    for i in range(len(first)):
        for j in range(len(v[first[i]])):
            if (num[v[first[i]][j]] == 2):
                start += 2
    ans = start
    anslst = []
    for i in range(len(first)):
        anslst.append(first[i])
    it2 = 0
    it1 = 0
    for ii in range(it):
        pos1 = random.randint(0, len(first) - 1)
        pos2 = random.randint(0, len(second) - 1)
        delta = 0
        for i in range(len(v[first[pos1]])):
            if (num[v[first[pos1]][i]] == 2):
                delta -= 2
            else:
                delta += 2
            if (v[first[pos1]][i] == second[pos2]):
                delta += 2
        for i in range(len(v[second[pos2]])):
            if (num[v[second[pos2]][i]] == 1):
                delta -= 2
            else:
                delta += 2
            if (v[second[pos2]][i] == first[pos1]):
                delta += 2
        if (delta < 0 or it2 > 0):
            start += delta
            it1 = 0
            first[pos1], second[pos2] = second[pos2], first[pos1]
            num[first[pos1]] = 1
            num[second[pos2]] = 2
            if (delta < 0):
                it2 = 0
            else:
                it2 -= 1
            if (start < ans):
                ans = start
                anslst = []
                for i in range(len(first)):
                    anslst.append(first[i])
        else:
            it1 += 1
            if (it1 == magic):
                it2 = magic1
    return set(anslst)

In [99]:
import time

def get_quality(graph, partition_part):
    if not (partition_part <= graph[0]) or abs(len(partition_part) - len(graph[0]) / 2) > 0.6:
        return -1
    other_part = set(graph[0]) - partition_part
    return sum(1 for edge in graph[1] if set(edge) <= partition_part or set(edge) <= other_part )

def run_all():
    filenames = ['add20.graph', 'cti.graph', 't60k.graph', 'm14b.graph']
    numit = [2220000, 2220000, 2600000, 77000]
    i = 0
    for filename in filenames:
        instance = read_instance(filename)
        print('Solving instance {}…'.format(filename), end='')
        time_start = time.monotonic()
        quality = get_quality(instance, variable_depth_local_search(instance, numit[i]))
        time_elapsed = time.monotonic()-time_start
        print(' done in {:.2} seconds with quality {}'.format(time_elapsed, quality))
        i += 1

In [100]:
run_all()

Solving instance add20.graph… done in 5.5e+01 seconds with quality 13066
Solving instance cti.graph… done in 5.6e+01 seconds with quality 94052
Solving instance t60k.graph… done in 5.3e+01 seconds with quality 178236
Solving instance m14b.graph… done in 6.2e+01 seconds with quality 2901188


## Выводы
Как видно, на первом и на четвертом тесте ответ улучшился, а на втором и третьем тесте ответ улучшить не получается. Я провел также серию запусков программы со случайными стартовыми долями, но это также не привело  улучшению ответа, значит, скорее всего, во втором и третьем тесте найденный ответ достаточно оптимален. Также с помощью нескольких запусков я подобрал оптимальные контанты для своего решения на первом и четвертом тесте.